In [1]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import skimage.io as io
import os, sys
from PIL import Image
import time

# Use second GPU -- change if you want to use a first one

# Add a path to a custom fork of TF-Slim
# Get it from here:
# https://github.com/warmspringwinds/models/tree/fully_conv_vgg
sys.path.append("/home/shou/network/tf-models/research/slim")

# Add path to the cloned library
sys.path.append("/home/shou/network/fcn/tf-image-segmentation")

checkpoints_dir = '/home/shou/network/checkpoints'
log_folder = '/home/shou/network/log_folder_fcn32s'

slim = tf.contrib.slim

from tf_image_segmentation.utils.tf_records import read_tfrecord_and_decode_into_image_annotation_pair_tensors
from tf_image_segmentation.models.fcn_16s import FCN_16s

from tf_image_segmentation.utils.pascal_voc import pascal_segmentation_lut

from tf_image_segmentation.utils.training import get_valid_logits_and_labels

from tf_image_segmentation.utils.augmentation import (distort_randomly_image_color,
                                                      flip_randomly_left_right_image_with_annotation,
                                                      scale_randomly_image_with_annotation_with_fixed_size_output)

In [2]:
image_train_size = [384, 384]
number_of_classes = 2
tfrecord_filename = '/home/shou/network/dataset/pascal_augmented_train_island.tfrecords'
pascal_voc_lut = pascal_segmentation_lut(number_of_classes)
class_labels = pascal_voc_lut.keys()

fcn_32s_checkpoint_path = '/home/shou/network/dataset/model_fcn32s.ckpt'

filename_queue = tf.train.string_input_producer(
    [tfrecord_filename], num_epochs=5)

image, annotation = read_tfrecord_and_decode_into_image_annotation_pair_tensors(filename_queue)

# Various data augmentation stages
image, annotation = flip_randomly_left_right_image_with_annotation(image, annotation)

# image = distort_randomly_image_color(image)

resized_image, resized_annotation = scale_randomly_image_with_annotation_with_fixed_size_output(image, annotation, image_train_size)


resized_annotation = tf.squeeze(resized_annotation)

image_batch, annotation_batch = tf.train.shuffle_batch( [resized_image, resized_annotation],
                                             batch_size=1,
                                             capacity=3000,
                                             num_threads=2,
                                             min_after_dequeue=1000)

upsampled_logits_batch, fcn_32s_variables_mapping = FCN_16s(image_batch_tensor=image_batch,
                                                           number_of_classes=number_of_classes,
                                                           is_training=True)


valid_labels_batch_tensor, valid_logits_batch_tensor = get_valid_logits_and_labels(annotation_batch_tensor=annotation_batch,
                                                                                     logits_batch_tensor=upsampled_logits_batch,
                                                                                    class_labels=class_labels)



cross_entropies = tf.nn.softmax_cross_entropy_with_logits(logits=valid_logits_batch_tensor,
                                                          labels=valid_labels_batch_tensor)

#cross_entropy_sum = tf.reduce_sum(cross_entropies)

cross_entropy_sum = tf.reduce_mean(cross_entropies)

pred = tf.argmax(upsampled_logits_batch, dimension=3)

probabilities = tf.nn.softmax(upsampled_logits_batch)

Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
Use the `axis` argument instead


In [3]:
with tf.variable_scope("adam_vars"):
    train_step = tf.train.AdamOptimizer(learning_rate=0.00000001).minimize(cross_entropy_sum)


#adam_optimizer_variables = slim.get_variables_to_restore(include=['adam_vars'])

# Variable's initialization functions
init_fn = slim.assign_from_checkpoint_fn(model_path=fcn_32s_checkpoint_path,
                                         var_list=fcn_32s_variables_mapping)

global_vars_init_op = tf.global_variables_initializer()

tf.summary.scalar('cross_entropy_loss', cross_entropy_sum)

merged_summary_op = tf.summary.merge_all()

summary_string_writer = tf.summary.FileWriter(log_folder)

# Create the log folder if doesn't exist yet
if not os.path.exists(log_folder):
     os.makedirs(log_folder)

#optimization_variables_initializer = tf.variables_initializer(adam_optimizer_variables)
    
#The op for initializing the variables.
local_vars_init_op = tf.local_variables_initializer()

combined_op = tf.group(local_vars_init_op, global_vars_init_op)

# We need this to save only model variables and omit
# optimization-related and other variables.
model_variables = slim.get_model_variables()
saver = tf.train.Saver(model_variables)

In [4]:
with tf.Session()  as sess:
    
    sess.run(combined_op)
    init_fn(sess)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    time_start=time.time()
    # Let's read off 3 batches just for example
    for i in xrange(486 * 5):
    
        cross_entropy, summary_string, _ = sess.run([ cross_entropy_sum,
                                                      merged_summary_op,
                                                      train_step ])

        summary_string_writer.add_summary(summary_string, 486 * 5 + i)
        
        print("step :" + str(i) + " Loss: " + str(cross_entropy))
        
        if i % 486 == 0:
            save_path = saver.save(sess, "/home/shou/network/dataset/model_fcn16s_final.ckpt")
            print("Model saved in file: %s" % save_path)
            
        
    coord.request_stop()
    coord.join(threads)
    time_end = time.time()
    print(str(time_end-time_start))
    
    save_path = saver.save(sess, "/home/shou/network/dataset/model_fcn16s_final.ckpt")
    print("Model saved in file: %s" % save_path)
    
summary_string_writer.close()

INFO:tensorflow:Restoring parameters from /home/shou/network/dataset/model_fcn32s.ckpt
step :0 Loss: 0.130845
Model saved in file: /home/shou/network/dataset/model_fcn16s_final.ckpt
step :1 Loss: 0.105321
step :2 Loss: 0.0776117
step :3 Loss: 0.106419
step :4 Loss: 0.111803
step :5 Loss: 0.0916703
step :6 Loss: 0.12547
step :7 Loss: 0.1232
step :8 Loss: 0.103953
step :9 Loss: 0.142505
step :10 Loss: 0.147072
step :11 Loss: 0.145642
step :12 Loss: 0.0613367
step :13 Loss: 0.143708
step :14 Loss: 0.149368
step :15 Loss: 0.0535257
step :16 Loss: 0.0621817
step :17 Loss: 0.168576
step :18 Loss: 0.107838
step :19 Loss: 0.102466
step :20 Loss: 0.21853
step :21 Loss: 0.146701
step :22 Loss: 0.230347
step :23 Loss: 0.103849
step :24 Loss: 0.169017
step :25 Loss: 0.131014
step :26 Loss: 0.168649
step :27 Loss: 0.177725
step :28 Loss: 0.117607
step :29 Loss: 0.139254
step :30 Loss: 0.0619673
step :31 Loss: 0.0728192
step :32 Loss: 0.183678
step :33 Loss: 0.163644
step :34 Loss: 0.0538853
step :3

step :323 Loss: 0.122542
step :324 Loss: 0.10092
step :325 Loss: 0.140345
step :326 Loss: 0.249168
step :327 Loss: 0.131779
step :328 Loss: 0.0789244
step :329 Loss: 0.078738
step :330 Loss: 0.141744
step :331 Loss: 0.158673
step :332 Loss: 0.167512
step :333 Loss: 0.127524
step :334 Loss: 0.0409013
step :335 Loss: 0.153578
step :336 Loss: 0.171723
step :337 Loss: 0.0523204
step :338 Loss: 0.136068
step :339 Loss: 0.0418608
step :340 Loss: 0.160776
step :341 Loss: 0.109327
step :342 Loss: 0.0882752
step :343 Loss: 0.185401
step :344 Loss: 0.0702564
step :345 Loss: 0.0520171
step :346 Loss: 0.183638
step :347 Loss: 0.142665
step :348 Loss: 0.15158
step :349 Loss: 0.106218
step :350 Loss: 0.152497
step :351 Loss: 0.117833
step :352 Loss: 0.155865
step :353 Loss: 0.202348
step :354 Loss: 0.154382
step :355 Loss: 0.107271
step :356 Loss: 0.0974207
step :357 Loss: 0.19206
step :358 Loss: 0.170244
step :359 Loss: 0.135693
step :360 Loss: 0.0781183
step :361 Loss: 0.194
step :362 Loss: 0.0633

step :646 Loss: 0.0285404
step :647 Loss: 0.0891457
step :648 Loss: 0.0712594
step :649 Loss: 0.20911
step :650 Loss: 0.200012
step :651 Loss: 0.302009
step :652 Loss: 0.0544589
step :653 Loss: 0.047568
step :654 Loss: 0.130164
step :655 Loss: 0.17175
step :656 Loss: 0.115925
step :657 Loss: 0.117491
step :658 Loss: 0.133351
step :659 Loss: 0.13336
step :660 Loss: 0.149394
step :661 Loss: 0.23286
step :662 Loss: 0.113439
step :663 Loss: 0.225383
step :664 Loss: 0.132231
step :665 Loss: 0.132537
step :666 Loss: 0.0818166
step :667 Loss: 0.098675
step :668 Loss: 0.130928
step :669 Loss: 0.0817765
step :670 Loss: 0.357762
step :671 Loss: 0.153639
step :672 Loss: 0.153727
step :673 Loss: 0.0758151
step :674 Loss: 0.0648266
step :675 Loss: 0.158712
step :676 Loss: 0.138761
step :677 Loss: 0.131191
step :678 Loss: 0.233919
step :679 Loss: 0.174041
step :680 Loss: 0.0735399
step :681 Loss: 0.11978
step :682 Loss: 0.195001
step :683 Loss: 0.137406
step :684 Loss: 0.157004
step :685 Loss: 0.172

step :971 Loss: 0.132261
step :972 Loss: 0.235683
Model saved in file: /home/shou/network/dataset/model_fcn16s_final.ckpt
step :973 Loss: 0.11998
step :974 Loss: 0.152946
step :975 Loss: 0.0838493
step :976 Loss: 0.112153
step :977 Loss: 0.302278
step :978 Loss: 0.0787115
step :979 Loss: 0.259138
step :980 Loss: 0.137562
step :981 Loss: 0.241011
step :982 Loss: 0.0850359
step :983 Loss: 0.095538
step :984 Loss: 0.18589
step :985 Loss: 0.0640356
step :986 Loss: 0.0720918
step :987 Loss: 0.127889
step :988 Loss: 0.148029
step :989 Loss: 0.264517
step :990 Loss: 0.25365
step :991 Loss: 0.238294
step :992 Loss: 0.167987
step :993 Loss: 0.175498
step :994 Loss: 0.172475
step :995 Loss: 0.0480551
step :996 Loss: 0.141083
step :997 Loss: 0.148905
step :998 Loss: 0.16309
step :999 Loss: 0.235332
step :1000 Loss: 0.100301
step :1001 Loss: 0.0399039
step :1002 Loss: 0.169208
step :1003 Loss: 0.0898172
step :1004 Loss: 0.124095
step :1005 Loss: 0.151779
step :1006 Loss: 0.163037
step :1007 Loss: 

step :1282 Loss: 0.209129
step :1283 Loss: 0.111241
step :1284 Loss: 0.133297
step :1285 Loss: 0.0945024
step :1286 Loss: 0.0842751
step :1287 Loss: 0.121979
step :1288 Loss: 0.23453
step :1289 Loss: 0.0310579
step :1290 Loss: 0.071081
step :1291 Loss: 0.182977
step :1292 Loss: 0.0993285
step :1293 Loss: 0.0819896
step :1294 Loss: 0.176304
step :1295 Loss: 0.153641
step :1296 Loss: 0.145204
step :1297 Loss: 0.153607
step :1298 Loss: 0.160272
step :1299 Loss: 0.16764
step :1300 Loss: 0.0790604
step :1301 Loss: 0.213595
step :1302 Loss: 0.259232
step :1303 Loss: 0.0996541
step :1304 Loss: 0.0692215
step :1305 Loss: 0.113765
step :1306 Loss: 0.171057
step :1307 Loss: 0.0490608
step :1308 Loss: 0.14377
step :1309 Loss: 0.0982296
step :1310 Loss: 0.0872344
step :1311 Loss: 0.229665
step :1312 Loss: 0.101496
step :1313 Loss: 0.106457
step :1314 Loss: 0.107017
step :1315 Loss: 0.0697514
step :1316 Loss: 0.0393984
step :1317 Loss: 0.172977
step :1318 Loss: 0.217972
step :1319 Loss: 0.194695
st

step :1592 Loss: 0.105079
step :1593 Loss: 0.114089
step :1594 Loss: 0.123845
step :1595 Loss: 0.133053
step :1596 Loss: 0.128761
step :1597 Loss: 0.155865
step :1598 Loss: 0.153922
step :1599 Loss: 0.130228
step :1600 Loss: 0.239088
step :1601 Loss: 0.0923748
step :1602 Loss: 0.118439
step :1603 Loss: 0.157889
step :1604 Loss: 0.0946632
step :1605 Loss: 0.305386
step :1606 Loss: 0.0961758
step :1607 Loss: 0.0796675
step :1608 Loss: 0.120356
step :1609 Loss: 0.288362
step :1610 Loss: 0.115409
step :1611 Loss: 0.176875
step :1612 Loss: 0.262545
step :1613 Loss: 0.0865687
step :1614 Loss: 0.0903167
step :1615 Loss: 0.0865153
step :1616 Loss: 0.192621
step :1617 Loss: 0.262701
step :1618 Loss: 0.0937341
step :1619 Loss: 0.234691
step :1620 Loss: 0.150529
step :1621 Loss: 0.200284
step :1622 Loss: 0.142848
step :1623 Loss: 0.0745257
step :1624 Loss: 0.142456
step :1625 Loss: 0.166273
step :1626 Loss: 0.111366
step :1627 Loss: 0.170418
step :1628 Loss: 0.241091
step :1629 Loss: 0.105554
ste

step :1904 Loss: 0.287075
step :1905 Loss: 0.100674
step :1906 Loss: 0.0549912
step :1907 Loss: 0.0644745
step :1908 Loss: 0.104136
step :1909 Loss: 0.155575
step :1910 Loss: 0.218437
step :1911 Loss: 0.0810082
step :1912 Loss: 0.138341
step :1913 Loss: 0.179157
step :1914 Loss: 0.1133
step :1915 Loss: 0.148515
step :1916 Loss: 0.0653238
step :1917 Loss: 0.0951457
step :1918 Loss: 0.0622249
step :1919 Loss: 0.0883135
step :1920 Loss: 0.170486
step :1921 Loss: 0.0410369
step :1922 Loss: 0.167852
step :1923 Loss: 0.134029
step :1924 Loss: 0.129541
step :1925 Loss: 0.153573
step :1926 Loss: 0.27637
step :1927 Loss: 0.250395
step :1928 Loss: 0.151859
step :1929 Loss: 0.0466458
step :1930 Loss: 0.0763548
step :1931 Loss: 0.156216
step :1932 Loss: 0.139754
step :1933 Loss: 0.0901253
step :1934 Loss: 0.155346
step :1935 Loss: 0.129846
step :1936 Loss: 0.146979
step :1937 Loss: 0.209537
step :1938 Loss: 0.223807
step :1939 Loss: 0.168266
step :1940 Loss: 0.120517
step :1941 Loss: 0.131836
step

step :2214 Loss: 0.0307201
step :2215 Loss: 0.0942315
step :2216 Loss: 0.17116
step :2217 Loss: 0.171665
step :2218 Loss: 0.118438
step :2219 Loss: 0.160566
step :2220 Loss: 0.0210082
step :2221 Loss: 0.110844
step :2222 Loss: 0.220304
step :2223 Loss: 0.212174
step :2224 Loss: 0.144891
step :2225 Loss: 0.0733954
step :2226 Loss: 0.200684
step :2227 Loss: 0.0512822
step :2228 Loss: 0.0724266
step :2229 Loss: 0.100532
step :2230 Loss: 0.0762746
step :2231 Loss: 0.278377
step :2232 Loss: 0.188678
step :2233 Loss: 0.0976646
step :2234 Loss: 0.192063
step :2235 Loss: 0.151936
step :2236 Loss: 0.159647
step :2237 Loss: 0.0964599
step :2238 Loss: 0.124928
step :2239 Loss: 0.0783706
step :2240 Loss: 0.130792
step :2241 Loss: 0.10158
step :2242 Loss: 0.099934
step :2243 Loss: 0.324675
step :2244 Loss: 0.193374
step :2245 Loss: 0.122075
step :2246 Loss: 0.311628
step :2247 Loss: 0.303652
step :2248 Loss: 0.116647
step :2249 Loss: 0.130761
step :2250 Loss: 0.0860373
step :2251 Loss: 0.186083
ste